In [1]:
import fantasyparse as fp
from feature_selector import FeatureSelector
# noinspection PyUnresolvedReferences
from fantasyalgo import best_subset
import datetime as dt
import numpy as np
import pandas as pd

In [2]:
# MORE DATA (SEVERAL TOURNAMENTS) AND BIGGER TIMEPERIOD + ADD WINRATE FEATURE
event = fp.Event(6588)
end = event.start_date - dt.timedelta(1)
start = end - dt.timedelta(150)
efilter = fp.EventFilter.ALL
rfilter = fp.RankFilter.ALL
delta = dt.timedelta(90)

In [3]:
players = event.get_players()
len(players)

80

In [4]:
costs_data = event.get_costs("majorChallengers.html")
len(costs_data)

80

In [5]:
for p in players:
    if p.name not in list(costs_data.keys()):
        name = f"{p.name}-"
        if name in list(costs_data.keys()):
            p.name = name
            print(p.name)

gxx-


In [7]:
actual = [p.calc_pts(event.key, rfilter) for p in players]
print(len(actual), actual[:5])

80 [-0.738, 1.325, 2.325, 11.137, 1.387]


In [8]:
dataframes = []
for p in players:
    dataframes.append(p.get_dataset(start, end, efilter, delta, efilter, rfilter))
len(dataframes)

TESTING: apex
GETTING: apex --> 6503
GETTING: apex --> 6345
GETTING: apex --> 6346
GETTING: apex --> 6347
GETTING: apex --> 6714
GETTING: apex --> 6140
GETTING: apex --> 6141
TOTAL EVENTS: 7
OK: 7
FAILED: 0
TESTING: dupreeh
GETTING: dupreeh --> 6503
GETTING: dupreeh --> 6345
GETTING: dupreeh --> 6346
GETTING: dupreeh --> 6347
GETTING: dupreeh --> 6714
GETTING: dupreeh --> 6140
GETTING: dupreeh --> 6141
TOTAL EVENTS: 7
OK: 7
FAILED: 0
TESTING: magisk
GETTING: magisk --> 6503
GETTING: magisk --> 6345
GETTING: magisk --> 6346
GETTING: magisk --> 6347
GETTING: magisk --> 6714
GETTING: magisk --> 6140
GETTING: magisk --> 6141
TOTAL EVENTS: 7
OK: 7
FAILED: 0
TESTING: zywoo
GETTING: zywoo --> 6503
GETTING: zywoo --> 6345
GETTING: zywoo --> 6346
GETTING: zywoo --> 6347
GETTING: zywoo --> 6714
GETTING: zywoo --> 6140
GETTING: zywoo --> 6141
TOTAL EVENTS: 7
OK: 7
FAILED: 0
TESTING: spinx
GETTING: spinx --> 6345
GETTING: spinx --> 6346
GETTING: spinx --> 6347
GETTING: spinx --> 6510
GETTING: spin

80

In [9]:
merged_data = pd.concat(dataframes)
merged_data.shape

(615, 17)

In [20]:
# SEPARATE PREDICTIONS
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# MODELS

def linear(x: np.ndarray, y: np.ndarray, vector: np.ndarray) -> float:
    model = LinearRegression()
    model.fit(x, y)
    return float(model.predict(vector))

def svr(x: np.ndarray, y: np.ndarray, vector: np.ndarray) -> float:
    model = SVR()
    model.fit(x, y)
    return float(model.predict(vector))

def knn(x: np.ndarray, y: np.ndarray, vector: np.ndarray, n: int) -> float:
    model = KNeighborsRegressor(n_neighbors = n)
    model.fit(x, y)
    return float(model.predict(vector))

def gradient(x: np.ndarray, y: np.ndarray, vector: np.ndarray) -> float:
    model = GradientBoostingRegressor()
    model.fit(x, y)
    return float(model.predict(vector))

def rforest(x: np.ndarray, y: np.ndarray, vector: np.ndarray, n: int) -> float:
    model = RandomForestRegressor(n_estimators = n)
    model.fit(x, y)
    return float(model.predict(vector))

def dtree(x: np.ndarray, y: np.ndarray, vector: np.ndarray) -> float:
    model = DecisionTreeRegressor()
    model.fit(x, y)
    return float(model.predict(vector))

test_a_target = [df['pts'] for df in dataframes]
test_a_stats = [p.get_stats(end - delta, end, efilter, rfilter) for p in players]
print(len(test_a_target), len(test_a_stats))

80 80


In [123]:
# Getting data to use
# TEST A
# FIRST: (avg rank, prize, rating, impact, kpr) (correlations)
# SECOND: (avg rank, dpr, kast, adr, kpr) (no rating and impact but all other stats)
# THIRD: (avg rank, rating, impact) (just 3 general params)
# FOURTH: (avg rank, dpr, kast, impact) (trying to abuse dpr and kast not correlating with impact)
# FIFTH: (avg rank, kast, impact)

test_a_data_first = [df[['avg rank', 'prize', 'rating', 'impact', 'kpr']]  for df in dataframes]
test_a_data_second = [df[['avg rank', 'dpr', 'kast', 'adr', 'kpr']] for df in dataframes]
test_a_data_third = [df[['avg rank', 'rating', 'impact']] for df in dataframes]
test_a_data_fourth = [df[['avg rank', 'dpr', 'kast', 'impact']] for df in dataframes]
test_a_data_fifth = [df[['avg rank', 'rating']] for df in dataframes]
print(len(test_a_data_first), len(test_a_data_second), len(test_a_data_third), len(test_a_data_fourth), len(test_a_data_fifth))

80 80 80 80 80


In [206]:
# TEST A: LINEAR
# (rating, dpr, kast, impact, adr, kpr)
pred1, pred2, pred3, pred4, pred5 = [], [], [], [], []

for i in range(80):

    Y = np.array(test_a_target[i])
    V = test_a_stats[i]

    V1 = np.array([[event.rank, event.prize, V[0], V[3], V[5]]])
    V2 = np.array([[event.rank, V[1], V[2], V[4], V[5]]])
    V3 = np.array([[event.rank, V[0], V[3]]])
    V4 = np.array([[event.rank, V[1], V[2], V[3]]])
    V5 = np.array([[event.rank, V[0]]])

    X1 = np.array(test_a_data_first[i].to_numpy())
    X2 = np.array(test_a_data_second[i].to_numpy())
    X3 = np.array(test_a_data_third[i].to_numpy())
    X4 = np.array(test_a_data_fourth[i].to_numpy())
    X5 = np.array(test_a_data_fifth[i].to_numpy())

    pred1.append(linear(X1, Y, V1))
    pred2.append(linear(X2, Y, V2))
    pred3.append(linear(X3, Y, V3))
    pred4.append(linear(X4, Y, V4))
    pred5.append(linear(X5, Y, V5))

costs = list(costs_data.values())

actual_best = best_subset(actual, costs)
best1 = best_subset(pred1, costs)
best2 = best_subset(pred2, costs)
best3 = best_subset(pred3, costs)
best4 = best_subset(pred4, costs)
best5 = best_subset(pred5, costs)

actual_res = sum(actual[i] for i in actual_best)
res1 = sum(actual[i] for i in best1)
res2 = sum(actual[i] for i in best2)
res3 = sum(actual[i] for i in best3)
res4 = sum(actual[i] for i in best4)
res5 = sum(actual[i] for i in best5)
print(f"ACTUAL: {actual_res}")
print(f"FIRST: {res1}")
print(f"SECOND: {res2}")
print(f"THIRD: {res3}")
print(f"FOURTH: {res4}")
print(f"FIFTH: {res5}")

ACTUAL: 59.6
FIRST: 10.177
SECOND: -0.839
THIRD: 7.225
FOURTH: 31.487
FIFTH: 11.687999999999999


In [208]:
# TEST A: GRADIENT
# (rating, dpr, kast, impact, adr, kpr)
pred1, pred2, pred3, pred4, pred5 = [], [], [], [], []

for i in range(80):

    Y = np.array(test_a_target[i])
    V = test_a_stats[i]

    V1 = np.array([[event.rank, event.prize, V[0], V[3], V[5]]])
    V2 = np.array([[event.rank, V[1], V[2], V[4], V[5]]])
    V3 = np.array([[event.rank, V[0], V[3]]])
    V4 = np.array([[event.rank, V[1], V[2], V[3]]])
    V5 = np.array([[event.rank, V[0]]])

    X1 = np.array(test_a_data_first[i].to_numpy())
    X2 = np.array(test_a_data_second[i].to_numpy())
    X3 = np.array(test_a_data_third[i].to_numpy())
    X4 = np.array(test_a_data_fourth[i].to_numpy())
    X5 = np.array(test_a_data_fifth[i].to_numpy())

    pred1.append(gradient(X1, Y, V1))
    pred2.append(gradient(X2, Y, V2))
    pred3.append(gradient(X3, Y, V3))
    pred4.append(gradient(X4, Y, V4))
    pred5.append(gradient(X5, Y, V5))

costs = list(costs_data.values())

actual_best = best_subset(actual, costs)
best1 = best_subset(pred1, costs)
best2 = best_subset(pred2, costs)
best3 = best_subset(pred3, costs)
best4 = best_subset(pred4, costs)
best5 = best_subset(pred5, costs)

actual_res = sum(actual[i] for i in actual_best)
res1 = sum(actual[i] for i in best1)
res2 = sum(actual[i] for i in best2)
res3 = sum(actual[i] for i in best3)
res4 = sum(actual[i] for i in best4)
res5 = sum(actual[i] for i in best5)
print(f"ACTUAL: {actual_res}")
print(f"FIRST: {res1}")
print(f"SECOND: {res2}")
print(f"THIRD: {res3}")
print(f"FOURTH: {res4}")
print(f"FIFTH: {res5}")

ACTUAL: 59.6
FIRST: 5.453
SECOND: 26.299
THIRD: 5.453
FOURTH: 22.772000000000002
FIFTH: 6.991


In [209]:
# TEST A: DTREE
# (rating, dpr, kast, impact, adr, kpr)
pred1, pred2, pred3, pred4, pred5 = [], [], [], [], []

for i in range(80):

    Y = np.array(test_a_target[i])
    V = test_a_stats[i]

    V1 = np.array([[event.rank, event.prize, V[0], V[3], V[5]]])
    V2 = np.array([[event.rank, V[1], V[2], V[4], V[5]]])
    V3 = np.array([[event.rank, V[0], V[3]]])
    V4 = np.array([[event.rank, V[1], V[2], V[3]]])
    V5 = np.array([[event.rank, V[0]]])

    X1 = np.array(test_a_data_first[i].to_numpy())
    X2 = np.array(test_a_data_second[i].to_numpy())
    X3 = np.array(test_a_data_third[i].to_numpy())
    X4 = np.array(test_a_data_fourth[i].to_numpy())
    X5 = np.array(test_a_data_fifth[i].to_numpy())

    pred1.append(dtree(X1, Y, V1))
    pred2.append(dtree(X2, Y, V2))
    pred3.append(dtree(X3, Y, V3))
    pred4.append(dtree(X4, Y, V4))
    pred5.append(dtree(X5, Y, V5))

costs = list(costs_data.values())

actual_best = best_subset(actual, costs)
best1 = best_subset(pred1, costs)
best2 = best_subset(pred2, costs)
best3 = best_subset(pred3, costs)
best4 = best_subset(pred4, costs)
best5 = best_subset(pred5, costs)

actual_res = sum(actual[i] for i in actual_best)
res1 = sum(actual[i] for i in best1)
res2 = sum(actual[i] for i in best2)
res3 = sum(actual[i] for i in best3)
res4 = sum(actual[i] for i in best4)
res5 = sum(actual[i] for i in best5)
print(f"ACTUAL: {actual_res}")
print(f"FIRST: {res1}")
print(f"SECOND: {res2}")
print(f"THIRD: {res3}")
print(f"FOURTH: {res4}")
print(f"FIFTH: {res5}")


ACTUAL: 59.6
FIRST: 16.317
SECOND: 32.525
THIRD: 26.299
FOURTH: 18.636000000000003
FIFTH: 22.196


In [211]:
# Getting data to use
# TEST B
# FIRST: (avg rank, prize, rating, impact, kpr) (correlations)
# SECOND: (avg rank, dpr, kast, adr, kpr) (no rating and impact but all other stats)
# THIRD: (avg rank, rating, impact) (just 3 general params)
# FOURTH: (avg rank, dpr, kast, impact) (trying to abuse dpr and kast not correlating with impact)
# FIFTH: (avg rank, kast, impact)

test_b_data = pd.concat(dataframes)
test_b_data_first = test_b_data[['avg rank', 'prize', 'rating', 'impact', 'kpr']]
test_b_data_second = test_b_data[['avg rank', 'dpr', 'kast', 'adr', 'kpr']]
test_b_data_third = test_b_data[['avg rank', 'rating', 'impact']]
test_b_data_fourth = test_b_data[['avg rank', 'dpr', 'kast', 'impact']]
test_b_data_fifth = test_b_data[['avg rank', 'rating']]
print(len(test_b_data_first), len(test_b_data_second), len(test_b_data_third), len(test_b_data_fourth), len(test_b_data_fifth))

test_b_target = test_b_data['pts']
test_b_stats = [p.get_stats(end - delta, end, efilter, rfilter) for p in players]
print(len(test_b_target), len(test_b_stats))

615 615 615 615 615
615 80


In [214]:
# TEST B: DTREE
# (rating, dpr, kast, impact, adr, kpr)
pred1, pred2, pred3, pred4, pred5 = [], [], [], [], []

for i in range(80):

    Y = np.array(test_b_target)
    V = test_b_stats[i]

    V1 = np.array([[event.rank, event.prize, V[0], V[3], V[5]]])
    V2 = np.array([[event.rank, V[1], V[2], V[4], V[5]]])
    V3 = np.array([[event.rank, V[0], V[3]]])
    V4 = np.array([[event.rank, V[1], V[2], V[3]]])
    V5 = np.array([[event.rank, V[0]]])

    X1 = np.array(test_b_data_first.to_numpy())
    X2 = np.array(test_b_data_second.to_numpy())
    X3 = np.array(test_b_data_third.to_numpy())
    X4 = np.array(test_b_data_fourth.to_numpy())
    X5 = np.array(test_b_data_fifth.to_numpy())

    pred1.append(dtree(X1, Y, V1))
    pred2.append(dtree(X2, Y, V2))
    pred3.append(dtree(X3, Y, V3))
    pred4.append(dtree(X4, Y, V4))
    pred5.append(dtree(X5, Y, V5))

costs = list(costs_data.values())

actual_best = best_subset(actual, costs)
best1 = best_subset(pred1, costs)
best2 = best_subset(pred2, costs)
best3 = best_subset(pred3, costs)
best4 = best_subset(pred4, costs)
best5 = best_subset(pred5, costs)

actual_res = sum(actual[i] for i in actual_best)
res1 = sum(actual[i] for i in best1)
res2 = sum(actual[i] for i in best2)
res3 = sum(actual[i] for i in best3)
res4 = sum(actual[i] for i in best4)
res5 = sum(actual[i] for i in best5)

print(f"ACTUAL: {actual_res}")
print(f"FIRST: {res1}")
print(f"SECOND: {res2}")
print(f"THIRD: {res3}")
print(f"FOURTH: {res4}")
print(f"FIFTH: {res5}")

ACTUAL: 59.6
FIRST: 14.75
SECOND: 19.29
THIRD: 27.275000000000002
FOURTH: 15.236000000000002
FIFTH: 14.55


In [215]:
# TEST B: LINEAR
# (rating, dpr, kast, impact, adr, kpr)
pred1, pred2, pred3, pred4, pred5 = [], [], [], [], []

for i in range(80):

    Y = np.array(test_b_target)
    V = test_b_stats[i]

    V1 = np.array([[event.rank, event.prize, V[0], V[3], V[5]]])
    V2 = np.array([[event.rank, V[1], V[2], V[4], V[5]]])
    V3 = np.array([[event.rank, V[0], V[3]]])
    V4 = np.array([[event.rank, V[1], V[2], V[3]]])
    V5 = np.array([[event.rank, V[0]]])

    X1 = np.array(test_b_data_first.to_numpy())
    X2 = np.array(test_b_data_second.to_numpy())
    X3 = np.array(test_b_data_third.to_numpy())
    X4 = np.array(test_b_data_fourth.to_numpy())
    X5 = np.array(test_b_data_fifth.to_numpy())

    pred1.append(linear(X1, Y, V1))
    pred2.append(linear(X2, Y, V2))
    pred3.append(linear(X3, Y, V3))
    pred4.append(linear(X4, Y, V4))
    pred5.append(linear(X5, Y, V5))

costs = list(costs_data.values())

actual_best = best_subset(actual, costs)
best1 = best_subset(pred1, costs)
best2 = best_subset(pred2, costs)
best3 = best_subset(pred3, costs)
best4 = best_subset(pred4, costs)
best5 = best_subset(pred5, costs)

actual_res = sum(actual[i] for i in actual_best)
res1 = sum(actual[i] for i in best1)
res2 = sum(actual[i] for i in best2)
res3 = sum(actual[i] for i in best3)
res4 = sum(actual[i] for i in best4)
res5 = sum(actual[i] for i in best5)

print(f"ACTUAL: {actual_res}")
print(f"FIRST: {res1}")
print(f"SECOND: {res2}")
print(f"THIRD: {res3}")
print(f"FOURTH: {res4}")
print(f"FIFTH: {res5}")

ACTUAL: 59.6
FIRST: 21.738999999999997
SECOND: 10.264
THIRD: 21.738999999999997
FOURTH: 21.738999999999997
FIFTH: 21.738999999999997
